<a href="https://colab.research.google.com/github/cjfghk5697/Classificiation_BMD/blob/main/No_Sam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install git+https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git


  Cloning https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git to /tmp/pip-req-build-1rkbnc4y
  Running command git clone -q https://github.com/cmpark0126/pytorch-polynomial-lr-decay.git /tmp/pip-req-build-1rkbnc4y


In [20]:
!pip3 install timm torchmetrics

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.optim as optim
import datetime
import csv

from torchvision.datasets import ImageFolder
from torchmetrics import AUROC, ROC

import gdown
import time
import logging
import random
import torch
from torch.autograd import Variable

from google.colab import files
from torch.optim.lr_scheduler import _LRScheduler
from torchsummary import summary
from pathlib import Path
from PIL import Image

import torchvision.models as models

from torch.utils.data.dataset import random_split

import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

import pandas as pd
from torch.utils.tensorboard import SummaryWriter
from torch_poly_lr_decay import PolynomialLRDecay

from tqdm import tqdm
import argparse
import albumentations as A
import cv2
from glob import glob
from albumentations.core.transforms_interface import ImageOnlyTransform
from matplotlib import pyplot as plt
import math
import timm
from torch.utils.data import Dataset, DataLoader

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
train_dir = '/content/drive/MyDrive/bmd_dataset/train'
model_root = './model/'

In [24]:
### https://github.com/davda54/sam

class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, rho=0.05, **kwargs):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho, **kwargs)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, **kwargs)
        self.param_groups = self.base_optimizer.param_groups

    @torch.no_grad()
    def first_step(self, zero_grad=False):
        grad_norm = self._grad_norm()
        for group in self.param_groups:
            scale = group["rho"] / (grad_norm + 1e-12)

            for p in group["params"]:
                if p.grad is None: continue
                e_w = p.grad * scale.to(p)
                p.add_(e_w)  # climb to the local maximum "w + e(w)"
                self.state[p]["e_w"] = e_w

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        for group in self.param_groups:
            for p in group["params"]:
                if p.grad is None: continue
                p.sub_(self.state[p]["e_w"])  # get back to "w" from "w + e(w)"

        self.base_optimizer.step()  # do the actual "sharpness-aware" update

        if zero_grad: self.zero_grad()

    @torch.no_grad()
    def step(self, closure=None):
        assert closure is not None, "Sharpness Aware Minimization requires closure, but it was not provided"
        closure = torch.enable_grad()(closure)  # the closure should do a full forward-backward pass

        self.first_step(zero_grad=True)
        closure()
        self.second_step()

    def _grad_norm(self):
        shared_device = self.param_groups[0]["params"][0].device  # put everything on the same device, in case of model parallelism
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2).to(shared_device)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        return norm

In [25]:
class CosineAnnealingWarmupRestarts(_LRScheduler):
    """
        optimizer (Optimizer): Wrapped optimizer.
        first_cycle_steps (int): First cycle step size.
        cycle_mult(float): Cycle steps magnification. Default: -1.
        max_lr(float): First cycle's max learning rate. Default: 0.1.
        min_lr(float): Min learning rate. Default: 0.001.
        warmup_steps(int): Linear warmup step size. Default: 0.
        gamma(float): Decrease rate of max learning rate by cycle. Default: 1.
        last_epoch (int): The index of last epoch. Default: -1.
    """
    
    def __init__(self,
                 optimizer : torch.optim.Optimizer,
                 first_cycle_steps : int,
                 cycle_mult : float = 1.,
                 max_lr : float = 0.1,
                 min_lr : float = 0.001,
                 warmup_steps : int = 0,
                 gamma : float = 1.,
                 last_epoch : int = -1
        ):
        assert warmup_steps < first_cycle_steps
        
        self.first_cycle_steps = first_cycle_steps # first cycle step size
        self.cycle_mult = cycle_mult # cycle steps magnification
        self.base_max_lr = max_lr # first max learning rate
        self.max_lr = max_lr # max learning rate in the current cycle
        self.min_lr = min_lr # min learning rate
        self.warmup_steps = warmup_steps # warmup step size
        self.gamma = gamma # decrease rate of max learning rate by cycle
        
        self.cur_cycle_steps = first_cycle_steps # first cycle step size
        self.cycle = 0 # cycle count
        self.step_in_cycle = last_epoch # step size of the current cycle
        
        super(CosineAnnealingWarmupRestarts, self).__init__(optimizer, last_epoch)
        
        # set learning rate min_lr
        self.init_lr()
    
    def init_lr(self):
        self.base_lrs = []
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.min_lr
            self.base_lrs.append(self.min_lr)
    
    def get_lr(self):
        if self.step_in_cycle == -1:
            return self.base_lrs
        elif self.step_in_cycle < self.warmup_steps:
            return [(self.max_lr - base_lr)*self.step_in_cycle / self.warmup_steps + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.max_lr - base_lr) \
                    * (1 + math.cos(math.pi * (self.step_in_cycle-self.warmup_steps) \
                                    / (self.cur_cycle_steps - self.warmup_steps))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.step_in_cycle = self.step_in_cycle + 1
            if self.step_in_cycle >= self.cur_cycle_steps:
                self.cycle += 1
                self.step_in_cycle = self.step_in_cycle - self.cur_cycle_steps
                self.cur_cycle_steps = int((self.cur_cycle_steps - self.warmup_steps) * self.cycle_mult) + self.warmup_steps
        else:
            if epoch >= self.first_cycle_steps:
                if self.cycle_mult == 1.:
                    self.step_in_cycle = epoch % self.first_cycle_steps
                    self.cycle = epoch // self.first_cycle_steps
                else:
                    n = int(math.log((epoch / self.first_cycle_steps * (self.cycle_mult - 1) + 1), self.cycle_mult))
                    self.cycle = n
                    self.step_in_cycle = epoch - int(self.first_cycle_steps * (self.cycle_mult ** n - 1) / (self.cycle_mult - 1))
                    self.cur_cycle_steps = self.first_cycle_steps * self.cycle_mult ** (n)
            else:
                self.cur_cycle_steps = self.first_cycle_steps
                self.step_in_cycle = epoch
                
        self.max_lr = self.base_max_lr * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [26]:
class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        
        self.degenerated_to_sgd = degenerated_to_sgd
        if isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], dict):
            for param in params:
                if 'betas' in param and (param['betas'][0] != betas[0] or param['betas'][1] != betas[1]):
                    param['buffer'] = [[None, None, None] for _ in range(10)]
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, buffer=[[None, None, None] for _ in range(10)])
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = group['buffer'][int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    elif self.degenerated_to_sgd:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    else:
                        step_size = -1
                    buffered[2] = step_size

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif step_size > 0:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss

In [27]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# Train

In [28]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Model list

In [29]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
seed_everything(42)

In [30]:
# all_densenet_models = timm.list_models('**')
# all_densenet_models

Model

In [31]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model('resnet34', pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.dropout = nn.Dropout(p=0.2)
        self.activation = nn.SiLU()
        self.classifier = nn.Linear(1000, 3)
    def forward(self, x):
        x=self.backbone(x)
        x=self.classifier(x)

        return x

Making a dataset

In [32]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(30),

])

dataset = ImageFolder(train_dir, transform=train_transform)

# normal: 0, osteopenia:1, osteoporosis:2

Train & Validation

In [33]:
net = Model()
net.train()
net = net.to(device)

criterion = nn.CrossEntropyLoss()

#base_optimizer = torch.optim.SGD  
#optimizer = SAM(net.parameters(), base_optimizer, lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(),lr=1e-3)
#optimizer = RAdam(net.parameters(), lr=1e-3, betas=(0.9, 0.999), weight_decay=1e-4)
#optimizer = torch.optim.AdamW(net.parameters(),lr=1e-3)

#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)    
scheduler = CosineAnnealingWarmupRestarts(optimizer,
                                          first_cycle_steps=60,
                                          cycle_mult=1.0,
                                          max_lr=0.005,
                                          min_lr=0.0001,
                                          warmup_steps=12,
                                          gamma=1.0)
batch_size = 32
train_size = int(0.7*len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
num_epochs=80

decay_steps = (len(train_dataset)//batch_size)*num_epochs

#scheduler = PolynomialLRDecay(optimizer, max_decay_steps=decay_steps, end_learning_rate=1e-6, power=0.9)


In [34]:
def save_model(model, acc, date_time, name):
    model_name = name + '.pth'
    model_path = Path(model_root + date_time)
    model_path.mkdir(parents=True, exist_ok=True)
    print('Saving model (Accuracy {:.2f}%) to {}'.format(acc*100, str(model_path / model_name)) )
    torch.save({'model_state_dict':model.state_dict(), 'acc':acc}, str(model_path / model_name) )

In [35]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs, auc=False, device=device,scheduler=scheduler):
    tz = datetime.timezone(datetime.timedelta(hours=9)) # Timezone infomation
    date_time = datetime.datetime.now(tz).strftime('%Y-%m-%d-%H-%M-%S')
    
    best_val_acc = 0
    best_train_acc = 0
    epoch_train_acc = 0
    
    for epoch in range(num_epochs + 1):
        print('\n------------------------')
        print('EPOCH {}/{}'.format(epoch, num_epochs))
        print('------------------------')
        
        if auc == True:
            auc_roc_metric = AUROC(num_classes=3, average=None)
            roc_metric = ROC(num_classes=3)

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            total_loss = 0
            total_size = 0
            total_corrects = 0
            
            # 학습 전 성능 확인
            if (epoch == 0) and (phase == 'train'):
                continue
            
            num_iteration = len(dataloaders_dict[phase])
            
            for idx, (inputs, labels) in enumerate(tqdm(dataloaders_dict[phase])):
                optimizer.zero_grad()



                with torch.set_grad_enabled(phase == 'train'):
                    inputs, labels = inputs.to(device), labels.to(device)
                    inputs, labels_a, labels_b, lam = mixup_data(inputs,labels)
                    inputs, labels_a, labels_b = map(Variable, (inputs, labels_a, labels_b))
                    outputs = net(inputs)
                    loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        #optimizer.first_step(zero_grad=True)
                        # sam optimizer second_steop
                        #criterion(net(inputs), labels).backward()
                        #optimizer.second_step(zero_grad=True)

                        optimizer.step()
                    epoch_loss = loss.item()
                    total_loss += epoch_loss

                    epoch_corrects = torch.sum(preds == labels.data)
                    epoch_acc = epoch_corrects.double() / inputs.size(0)
                    total_corrects += epoch_corrects
                    total_size += inputs.size(0)
                    
                    if phase == 'train':
                        print('{} [{}/{}] LOSS: {:.4f} ACC: {:.4f}'.format(phase, idx+1, num_iteration, epoch_loss, epoch_acc))

                    if (phase == 'val') & (auc == True):
                        auc_roc_metric(outputs, labels)
                        roc_metric(outputs, labels)
            
            epoch_loss_avg = total_loss / num_iteration
            epoch_acc_avg = total_corrects / total_size
            scheduler.step()
            if phase == 'train':
                if best_train_acc < epoch_acc_avg:
                    best_train_acc = epoch_acc_avg
                print('{} LOSS: {:.4f} ACC: {:.4f} BEST ACC: {:.4f}'.format(phase, epoch_loss_avg, epoch_acc_avg, best_train_acc))

            if phase == 'val':
                if (epoch_acc_avg == best_val_acc) & (epoch_acc_avg == best_train_acc):
                    save_model(net, epoch_acc_avg, date_time, 'best')
                elif best_val_acc < epoch_acc_avg:
                    best_val_acc = epoch_acc_avg
                    print('Best Validation Accuracy: {:.4f}'.format(epoch_acc_avg))
                    save_model(net, epoch_acc_avg, date_time, 'best')

                print('{} LOSS: {:.4f} ACC: {:.4f} BEST ACC: {:.4f}\n'.format(phase, epoch_loss_avg, epoch_acc_avg, best_val_acc))

                if auc == True:
                    for auroc, fpt, tpr, thresholds in zip(auc_roc_metric.compute(), *roc_metric.compute()):
                        print('auc: {:5.2f}'.format(auroc* 100))
                        size = min(len(fpt), len(tpr))
                        plt.plot(fpt[:size].cpu(), tpr[:size].cpu())
                        plt.show()
                        plt.cla()

In [ ]:


num_workers = 64
view_auc = False




train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=num_workers)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=num_workers)

dataloaders_dict = {"train": train_loader, "val": val_loader}

train_model(net, dataloaders_dict, criterion, optimizer, num_epochs, view_auc,scheduler=scheduler)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 64 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



------------------------
EPOCH 0/80
------------------------


100%|██████████| 29/29 [00:09<00:00,  3.04it/s]


Best Validation Accuracy: 0.3944
Saving model (Accuracy 39.44%) to model/2021-12-16-23-06-26/best.pth
val LOSS: 1.5381 ACC: 0.3944 BEST ACC: 0.3944


------------------------
EPOCH 1/80
------------------------


  2%|▏         | 1/66 [00:15<16:28, 15.21s/it]

...

100%|██████████| 66/66 [00:17<00:00, 10.30it/s]

train [65/66] LOSS: 1.0598 ACC: 0.5312
train [66/66] LOSS: 1.0826 ACC: 0.4500


100%|██████████| 66/66 [00:18<00:00,  3.50it/s]


train LOSS: 1.0476 ACC: 0.4005 BEST ACC: 0.4276


  0%|          | 0/29 [00:00<?, ?it/s]